# Part I

## Top 10 Lists from criterion.com
 https://www.criterion.com/current/category/8-top-10-lists
- Loop through all the links and extract the 24 top 10 lists on the webpage and store in a csv file `top10lists.csv`.
- Read the csv file into a pandas DataFrame.
- Which films and film directors make the top 10 lists most frequently?

__Exercises__: <br>
(a) Extend the program above to extract the 24 top 10 lists on the first page into a pandas dataframe; <br>
 - Done

(b) Extend part (a) to extract all the top 10 lists on the website (195)


In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

webpage = 'https://www.criterion.com/current/category/8-top-10-lists'
page = urlopen(webpage)
soup = BeautifulSoup(page, 'html.parser')

# find all the links
# extract all the <a> with class attribute "more-article more-article-small" 
all_links = [div.a for div in 
        soup.find_all("div", class_="more-article more-article-small")]
all_links.extend([div.a for div in soup.find_all("div", class_="more-article more-article-small is-newly-loaded is-ready-to-animate-in is-in")])
print('The number of links extracted:', len(all_links))

The number of links extracted: 24


In [2]:
# import re
import pandas as pd

# create a dict to store data
d = {'actor': [], 'author':[], 'ranks': [], 'title': []} 
for link in all_links:
    # open the webpage
    p1 = urlopen(link['href'])
    # parse the html
    s1 = BeautifulSoup(p1,'html.parser')
    
    # extract ranking of each movie using css selection
    ranks = [r.text for r in s1.select('p.count')]
    # clean up ranking, since top 10 sometimes has ties
    for i in range(len(ranks)):
        p = re.compile('\(tie\)')
        ranks[i] = int(p.sub('',ranks[i]).strip() or 0)
        if ranks[i] == 0: ranks[i] = ranks[i-1]

    # extract director, movie, actor name
    author = [a.text for a in s1.select('p.who')]
    title = [t.text for t in s1.select('div.what h3')]

    actor = s1.select('div.content-block h1') 
    str = (actor[0].text + ',')*len(ranks)
    actor = str.split(',')[0:len(ranks)]
    actor = [s[:-8] for s in actor]
      
    # add into the dict 
    d['actor'].extend(actor)
    d['author'].extend(author)
    d['ranks'].extend(ranks)
    d['title'].extend(title)

# store dict in pandas data frame
df1 = pd.DataFrame(d)
df1.head(20)

NameError: name 're' is not defined

In [ ]:
counts = df1.author.value_counts() # 
counts[0:10].plot(kind='barh',x = 'Frequency of Appearance on Top 10 Lists')

In [ ]:
df1.title.value_counts()[0:10].plot(kind='barh', x = 'Frequency of Appearance on Top 10 Lists')

# Part II

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

sns.set(style = "ticks", color_codes = True)

In [ ]:
# import data
# The data can be found in https://github.com/rudeboybert/JSE_OkCupid/blob/master/profiles.csv.zip
df = pd.read_csv('./profiles.csv')

In [ ]:
# see whats the catagories are
print(df.columns)

In [ ]:
print(df.head())

In [ ]:
print(df[0:10])

In [ ]:
print(df.hist('age'))

In [ ]:
print(df.hist('age', bins = range(18, 110, 1)))

In [ ]:
df.age.unique()
# The 110 109 data looks strange

In [ ]:
df.loc[df[ 'age' ] == 110]

In [ ]:
df.loc[df[ 'age' ] == 109]

In [ ]:
# Mean of age and the 95% level 
print(np.mean(df['age']))
print(np.percentile(df['age'], 95))

In [ ]:
any(g < 18 for g in df['age'])

In [ ]:
all(g >= 18 for g in df['age'])

In [ ]:
females = df.loc[df['sex'] == 'f']
males = df.loc[df['sex'] == 'm']
males['age'].hist(bins=range(10, 100, 5), alpha = 0.4, color='green', label = 'M')
females['age'].hist(bins=range(10, 100, 5), alpha = 0.4, color='red', label = 'F')

In [ ]:
df.drugs.unique()

In [ ]:
sns.catplot(x = 'drugs', kind = 'count', hue = 'sex', data = df)